In [11]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from vgg import *

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, TerminateOnNaN, TensorBoard
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os
import copy
import sys
from loguru import logger


logger.debug('All modules imported')

2020-02-09 03:00:42.006 | DEBUG    | __main__:<module>:28 - All modules imported


In [2]:
# current working directory
cur_dir = os.getcwd()

# the data is located in this data_dir
data_dir = os.path.join(cur_dir, 'Dataset')

# the output model and the graph is saved in this 'output_dir'
output_dir = os.path.join(cur_dir, 'Output')

# initialize the data and labels
logger.debug("[INFO] loading images...")
data = []
labels = []

# grab the image paths and shuffle them
# for reproducible results, use a constant seed for shuffling
imagePaths = sorted(list(paths.list_images(os.path.join(data_dir, 'Data') )))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
    
    # load the image
    image = cv2.imread(imagePath)
    
    # resize it
    image = cv2.resize(image, (32, 32))
    
    # append to the data list
    data.append(image)

    # extract the class label from the image path and update the labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)


logger.debug('[INFO] data loaded complete...')

2020-02-09 02:40:28.424 | DEBUG    | __main__:<module>:11 - [INFO] loading images...
2020-02-09 02:40:49.295 | DEBUG    | __main__:<module>:38 - [INFO] data loaded complete...


In [3]:
data_loaded = copy.deepcopy(data)
labels_loaded = copy.deepcopy(labels)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data_loaded, dtype="float") / 255.0
labels = np.array(labels_loaded)

# Binarize labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# Randomly split the data into test and train sets (25% test and 75% train)
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)



In [4]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

# initialize our VGG-like Convolutional Neural Network
model = SmallVGGNet.build(width=32, height=32, depth=3,
    classes=len(lb.classes_))








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.005
EPOCHS = 50
BS = 32

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='VGG_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=20)
terminate_on_nan = TerminateOnNaN()
tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

callbacks = [model_checkpoint, tb,terminate_on_nan]

# initialize the model and optimizer
print("[INFO] training network...")
# opt = SGD(lr=INIT_LR, momentum=0.5, decay=INIT_LR / EPOCHS)

opt = Adam(lr=INIT_LR, beta_1=0.9, beta_2=0.999, amsgrad=False)



model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS,callbacks=callbacks)



# Save the model locally for use later
model_path = os.path.join(output_dir, 'path_to_my_VGG_model.h5')
model.save(model_path)

[INFO] training network...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
118/118 [==============================] - 45s 381ms/step - loss: 1.8224 - acc: 0.4637 - val_loss: 4.6939 - val_acc: 0.3027
Epoch 2/50
118/118 [==============================] - 30s 250ms/step - loss: 1.1517 - acc: 0.6417 - val_loss: 1.7227 - val_acc: 0.5483
Epoch 3/50
118/118 [==============================] - 30s 251ms/step - loss: 0.9708 - acc: 0.7092 - val_loss: 2.7639 - val_acc: 0.4390
Epoch 4/50
118/118 [==============================] - 30s 258ms/step - loss: 0.7739 - acc: 0.7489 - val_loss: 0.9120 - val_acc: 0.7678
Epoch 5/50
118/118 [==============================] - 32s 271ms/step - loss: 0.6875 - acc: 0.7844 - val_loss: 1.0122 - val_acc: 0.6918
Epoch 6/50
118/118 [==============================] - 37s 315ms/step - loss: 0.6704 - acc: 0.7913 - val_loss: 0.6397 - val_acc: 0.8296
Epoch 7/50
118/118 [==============================] - 31s 259ms/st

KeyboardInterrupt: 

In [ ]:
# evaluate the network
logger.debug('Making predictions and evaluating the trained model.')
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=lb.classes_))

In [ ]:
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(os.path.join(output_dir,'smallvggnet_plot.png'))